- avoir ces 3 baselines qui marchent
- faire le bench pour les 2 autres papiers
- réfléchir à intégrer des éléments du cours (au moins insister sur IPTW?) -> évaluer efficacité des traintements

In [2]:
import pickle
import os
import numpy as np
import pandas as pd

# Assuming the script is located in the same directory as the 'results' folder
folder_path = os.path.join('Counterfactual-Recurrent-Network-master', 'results')
file_name = 'new_cancer_sim_2_2.p'

path = os.path.join(folder_path, file_name)

with open(path, 'rb') as f:
    results = pickle.load(f)

In [3]:
results.keys()

dict_keys(['chemo_coeff', 'radio_coeff', 'num_time_steps', 'training_data', 'validation_data', 'test_data', 'test_data_factuals', 'test_data_seq', 'scaling_data', 'window_size'])

In [4]:
results['test_data_factuals']['chemo_dosage'].shape

(1000, 60)

In [5]:
results['training_data'].keys()

dict_keys(['cancer_volume', 'chemo_dosage', 'radio_dosage', 'chemo_application', 'radio_application', 'chemo_probabilities', 'radio_probabilities', 'sequence_lengths', 'patient_types'])

In [6]:
cancer_volume = results['training_data']['cancer_volume']
print(cancer_volume.shape)
chemo_dosage = results['training_data']['chemo_dosage']
print(chemo_dosage.shape)
radiation_dosage = results['training_data']['radio_dosage']
print(radiation_dosage.shape)
patient_type = results['training_data']['patient_types']
print(patient_type.shape)

(10000, 60)
(10000, 60)
(10000, 60)
(10000,)


# Random Forest

### Model creation and training

In [7]:
# # preprocess each one of the series and concatenate those to the global dataframe

# data_train = pd.DataFrame(columns=['cancer_volume', 'chemo_dosage', 'radio_dosage', 'patient_type'])

# for i in range(cancer_volume.shape[0]):
#     sub_df = pd.DataFrame(columns=['cancer_volume', 'chemo_dosage', 'radio_dosage', 'patient_type'])
#     sub_df['cancer_volume'] = cancer_volume[i]
#     sub_df['chemo_dosage'] = chemo_dosage[i]
#     sub_df['radio_dosage'] = radiation_dosage[i]
#     sub_df['patient_type'] = patient_type[i]
#     sub_df['cancer_lag_1'] = sub_df['cancer_volume'].shift(1)
#     sub_df.dropna(inplace=True)
#     data_train = data_train.append(sub_df)

# data_test = pd.DataFrame(columns=['cancer_volume', 'chemo_dosage', 'radio_dosage', 'patient_type'])

# for i in range(results['test_data_factuals']['cancer_volume'].shape[0]):
#     sub_df = pd.DataFrame(columns=['cancer_volume', 'chemo_dosage', 'radio_dosage', 'patient_type'])
#     sub_df['cancer_volume'] = results['test_data_factuals']['cancer_volume'][i]
#     sub_df['chemo_dosage'] = results['test_data_factuals']['chemo_dosage'][i]
#     sub_df['radio_dosage'] = results['test_data_factuals']['radio_dosage'][i]
#     sub_df['patient_type'] = results['test_data_factuals']['patient_types'][i]
#     sub_df['cancer_lag_1'] = sub_df['cancer_volume'].shift(1)
#     sub_df.dropna(inplace=True)
#     data_test = data_test.append(sub_df)

# data_train.to_csv('data_train.csv', index=False)
# data_test.to_csv('data_test.csv', index=False)


In [8]:
data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

lags = 1

# Define features and target variable
features = ['chemo_dosage', 'radio_dosage'] + [f'cancer_lag_{i}' for i in range(1, lags + 1)]
target = 'cancer_volume'

X_train, y_train = data_train[features], data_train[target]
X_test, y_test = data_test[features], data_test[target]

# Create and train the RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict the next values using the test set
y_pred = rf_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_test[:120], label='Test')
plt.plot(y_pred[:120], label='Predicted')
plt.legend()
plt.show()


### Accuracy benchmarking for counterfactuals simulation

In [ ]:
# TODO: add counterfactuals benchmaring

# LSTM

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have already processed your data into tensors
cancer_volume = results['training_data']['cancer_volume']
chemo_dosage = results['training_data']['chemo_dosage']
radiation_dosage = results['training_data']['radio_dosage']
patient_type = results['training_data']['patient_types']

# Concatenate the covariates along the feature dimension
covariates = torch.stack([chemo_dosage, radiation_dosage, patient_type], dim=2)

# Combine all the data into a single tensor
all_data = torch.stack([cancer_volume, covariates], dim=2)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Take the output from the last time step
        return output

# Set hyperparameters
input_size = 4  # cancer_volume + chemo_dosage + radiation_dosage + patient_type
hidden_size = 64
num_layers = 2
output_size = 1  # Predicting cancer_volume

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to PyTorch tensors
all_data = torch.FloatTensor(all_data)
cancer_volume = torch.FloatTensor(cancer_volume)

# Define the DataLoader
dataset = TensorDataset(all_data, cancer_volume)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Training loop
losses = []
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    losses.append(loss.item())


NameError: name 'results' is not defined